# Test scenario generation

In [ ]:
import os
from typing import NamedTuple
import pickle
import matplotlib.pyplot as plt
from tikzplotlib import save
import numpy as np
import ot
from tqdm import tqdm_notebook as tqdm
from scipy.spatial import distance
from scipy.stats import ttest_ind
from sklearn.model_selection import train_test_split
from domain_model import DocumentManagement, StateVariable
from stats import KDE

In [ ]:
OVERWRITE = False

# Lead vehicle decelerating

## Load data

In [ ]:
leaddec = DocumentManagement(os.path.join("data", "5_scenarios", "lead_braking2.json"))
print("Number of lead vehicle decelerating scenarios: {:d}"
      .format(len(leaddec.collections["scenario"])))

In [ ]:
n = 50
data1 = np.zeros((len(leaddec.collections["scenario"]), n+1))
for i, key in enumerate(leaddec.collections["scenario"]):
    scenario = leaddec.get_item("scenario", key)
    data1[i, n] = scenario.get_duration()
    time = np.linspace(0, data1[i, n], n) + scenario.get_tstart()
    data1[i, :n] = scenario.get_state(scenario.get_actor_by_name("target vehicle"),
                                      StateVariable.LON_TARGET, time)[:, 0]
    
# Do the weighting
weights1 = np.zeros(data1.shape[1])
weights1[:n] = 2 / (np.mean(np.std(data1[:, :n], axis=0))*n)
weights1[n] = 1 / np.std(data1[:, :n])
data1 = data1 * weights1

In [ ]:
Nz = len(data1) // 4
N = len(data1) - Nz
Nw = 10000
print(N, Nz, Nw)

## Define method

In [ ]:
def score(generated, data):
    return ot.emd2([], [], distance.cdist(generated, data))

In [ ]:
def naive_approach(data, n=None):
    i = np.random.choice(np.arange(len(data)), len(data) if n is None else n)
    return data[i, :]

def new_approach(data, d=3, n=None):
    if n is None:
        n = len(data)
    generated = np.zeros((n, data.shape[1]))
    mean = np.mean(data, axis=0)
    u,s,v = np.linalg.svd(data-mean, full_matrices=False)
    k = KDE(u[:, :d], scaling=True)
    # k.set_bandwidth(k.silverman())
    k.compute_bandwidth(max_bw=k.silverman())
    for i in range(n):
        pars = k.sample()[0]
        generated[i] = np.dot(pars*s[:d], v[:d]) + mean
    return generated

## Test for different values of d

In [ ]:
def test_method(name, data, dmin, dmax, beta=1, seed=0, overwrite=False):
    filename = os.path.join("data", "3_test", 
                            "{:s}_d{:d}-{:d}_seed{:d}.p".format(name, dmin, dmax, seed))
    if os.path.exists(filename) and not overwrite:
        with open(filename, "rb") as file:
            scores1, scores2 = pickle.load(file)
    else:
        scores1 = np.zeros(dmax-dmin+2)
        scores2 = np.zeros_like(scores1)
        
        np.random.seed(seed)
        indexa, indexb = train_test_split(np.arange(len(data)), test_size=Nz, 
                                          random_state=seed)
        dataa, datab = data[indexa, :], data[indexb, :]
        
        generated = naive_approach(dataa, n=Nw)
        scores1[0] = score(generated, datab)
        scores2[0] = score(generated, dataa)
        
        for i in range(dmin, dmax+1):
            generated = new_approach(dataa, d=i, n=Nw)
            scores1[i-dmin+1] = score(generated, datab)
            scores2[i-dmin+1] = score(generated, dataa)
        with open(filename, "wb") as file:
            pickle.dump((scores1, scores2), file)
    
    combined_score = (1+beta)*scores1-beta*scores2
    for i in range(len(scores1)):
        print("q={:2d}: {:7.4f} {:7.4f} {:7.4f}".format(i, scores1[i], scores2[i], combined_score[i]),
              end="")
        if combined_score[i] == np.min(combined_score):
            print("  *")
        else:
            print()

In [ ]:
# Show singular values
def plot_show_svd(data):
    s_max = 8
    mean = np.mean(data, axis=0)
    _, s, _ = np.linalg.svd(data-mean, full_matrices=False)
    plt.semilogy(np.arange(len(s))+1, s, '.')
    plt.xlim(0, s_max+.5)
    plt.ylim(s[s_max+1], s[0]*2)
    plt.xlabel("Index")
    plt.ylabel("Singular value")
    
    print(" #  Explained variance [%]")
    for i in range(1, 9):
        print("{:2d} {:.1f} %".format(i, np.sum(s[:i]**2) / np.sum(s**2) * 100))
plot_show_svd(data1)

In [ ]:
test_method("lead_vehicle_decelerating_loocv", data1, 1, 8, beta=.48, seed=0, overwrite=OVERWRITE)

## Test multiple times

In [ ]:
def test_multiple_times(name, data, dmin, dmax, seed=0, overwrite=False):
    filename = os.path.join("data", "3_test", 
                            "{:s}_rep_d{:d}-{:d}_seed{:d}".format(name, dmin, dmax, seed))
    if os.path.exists(filename) and not overwrite:
        with open(filename, "rb") as file:
            return pickle.load(file)
        
    nrepeat = 50
    np.random.seed(seed)
    scores1 = np.zeros((nrepeat, dmax-dmin+2))
    scores2 = np.zeros_like(scores1)
    
    for i in tqdm(range(nrepeat)):
        indexa, indexb = train_test_split(np.arange(N+Nz), test_size=Nz,
                                          random_state=nrepeat*seed+i)
        dataa, datab = data[indexa, :], data[indexb, :]

        # Using the default.
        generated = naive_approach(dataa, n=Nw)
        scores1[i, 0] = score(generated, datab)
        scores2[i, 0] = score(generated, dataa)

        # Using approach with different d values.
        for d in range(dmin, dmax+1):
            generated = new_approach(dataa, d=d, n=Nw)
            scores1[i, d-dmin+1] = score(generated, datab)
            scores2[i, d-dmin+1] = score(generated, dataa)
    
    with open(filename, "wb") as file:
        pickle.dump((scores1, scores2), file)
    
    return scores1, scores2

In [ ]:
def boxplots(scores, ylabel="metric", ax=None):
    if ax is None:
        _, ax = plt.subplots()
    ax.boxplot(scores)
    ax.set_xticks(np.arange(1, dmax-dmin+3))
    ax.set_xticklabels(["Training set\n{:.3f}".format(np.median(scores[:, 0]))] +
                       ["d={:d}\n{:.3f}".format(d, np.median(scores[:, d-dmin+1])) 
                        for d in range(dmin, dmax+1)])
    if ylabel == "metric":
        ax.set_ylabel(r"$M(\mathcal{W},\mathcal{Z},\mathcal{X})$")
    elif ylabel == "many":
        ax.set_ylabel(r"$W(\hat{f},f)$")
    elif ylabel == "self":
        ax.set_ylabel(r"$W(\mathcal{W}, \mathcal{X})$")
    elif ylabel == "penalty":
        ax.set_ylabel(r"$W(\mathcal{W}, \mathcal{Z}) - W(\mathcal{W}, \mathcal{X})$")
    elif ylabel == "wasserstein":
        ax.set_ylabel(r"$W(\mathcal{W}, \mathcal{Z})$")

In [ ]:
dmin, dmax = 2, 5
beta1 = .25
s1, s2 = test_multiple_times("lead_vehicle_decelerating_loocv", data1, dmin, dmax, overwrite=OVERWRITE)
boxplots(s1 + beta1*(s1-s2))

## Validation of scoring measure

In [ ]:
def validation_scoring(name, data, dreal, dmin, dmax, overwrite=False):
    filename = os.path.join("data", "3_test", 
                            "{:s}_val_d{:d}-{:d}_dreal{:d}".format(name, dmin, dmax, dreal))
    if os.path.exists(filename) and not overwrite:
        with open(filename, "rb") as file:
            return pickle.load(file)
    
    nrepeat = 50
    np.random.seed(0)
    
    old_few = np.zeros((nrepeat, dmax-dmin+2))
    old_many = np.zeros_like(old_few)
    self_few = np.zeros_like(old_few)
    
    for i in tqdm(range(nrepeat)):
        data_new = new_approach(data, d=dreal, n=N)
        data_few = new_approach(data, d=dreal, n=Nz)
        data_many = new_approach(data, d=dreal, n=Nw)

        # Using the default.
        generated= naive_approach(data_new, n=Nw)
        old_few[i, 0] = score(generated, data_few)
        old_many[i, 0] = score(generated, data_many)
        self_few[i, 0] = score(generated, data_new)

        # Using new approach with different d
        for d in range(dmin, dmax+1):
            generated = new_approach(data_new, d=d, n=Nw)
            old_few[i, d-dmin+1] = score(generated, data_few)
            old_many[i, d-dmin+1] = score(generated, data_many)
            self_few[i, d-dmin+1] = score(generated, data_new)
            
    with open(filename, "wb") as file:
        pickle.dump((old_few, old_many, self_few), file)
    
    return old_few, old_many, self_few

In [ ]:
def plot_correlation(old_many, old_few, self_few):
    beta = np.linspace(0, 1, 100)
    correlation = np.zeros(len(beta))
    for i in range(len(beta)):
        new_score = old_few + beta[i]*(old_few - self_few)
        correlation[i] = np.corrcoef(np.median(old_many, axis=0),
                                     np.median(new_score, axis=0))[0, 1]
        
    _, ((ax11, ax12), (ax21, ax22)) = plt.subplots(2, 2, figsize=(12, 8))
    boxplots(old_many, "many", ax=ax11)
    boxplots(old_few, "wasserstein", ax=ax12)
    boxplots(old_few - self_few, "penalty", ax=ax21)
    ax22.plot(beta, correlation)
    ax22.set_xlabel(r"$\beta$")
    ax22.set_ylabel(r"Correlation medians of $W(\hat{f},f)$ and "+
                    r"$M(\mathcal{W}, \mathcal{Z}, \mathcal{X})$")
    ax22.set_title("Max correlation: {:.3f} at beta={:.2f}".format(np.max(correlation), 
                                                                   beta[np.argmax(correlation)]))
    plt.tight_layout()

In [ ]:
dreal = 2
old_few1, old_many1, self_few1 = validation_scoring("lead_vehicle_decelerating_loocv", data1, dreal, 
                                                    dmin, dmax, overwrite=OVERWRITE)
plot_correlation(old_many1, old_few1, self_few1)

# Cut-in

## Load data

y(t) - (1) speed of other car, (2) y of other car

theta - (1) initial ego vehicle speed, (2) initial longitudinal position other car

In [ ]:
cutins = DocumentManagement(os.path.join("data", "5_scenarios", "cut_in_scenarios2.json"))
print("Number of cut-in scenarios: {:d}".format(len(cutins.collections["scenario"])))

In [ ]:
n = 50
data2 = np.zeros((len(cutins.collections["scenario"]), 2*n+3))
for i, key in enumerate(cutins.collections["scenario"]):
    scenario = cutins.get_item("scenario", key)
    data2[i, 2*n] = scenario.get_duration()
    time = np.linspace(0,  data2[i, 2*n], n) + scenario.get_tstart()
    data2[i, :n] = scenario.get_state(scenario.get_actor_by_name("target vehicle"),
                                      StateVariable.LON_TARGET, time)[:, 0]
    data2[i, n:2*n] = scenario.get_state(scenario.get_actor_by_name("target vehicle"),
                                         StateVariable.LAT_TARGET, time)
    data2[i, 2*n+1] = scenario.get_state(scenario.get_actor_by_name("ego vehicle"),
                                         StateVariable.SPEED, scenario.get_tstart())
    data2[i, 2*n+2] = scenario.get_state(scenario.get_actor_by_name("target vehicle"),
                                         StateVariable.LON_TARGET, scenario.get_tstart())[1]

# Do the weighting
weights2 = np.zeros(data2.shape[1])
weights2[:n] = 1 / (np.mean(np.std(data2[:, :n], axis=0))*n)
weights2[n:2*n] = 1 / (np.mean(np.std(data2[:, n:2*n], axis=0))*n)
weights2[2*n:] = 1 / np.std(data2[:, 2*n:], axis=0)
data2 = data2 * weights2
weights2

In [ ]:
Nz = len(data2) // 4
N = len(data2) - Nz
Nw = 5000
print(N, Nz, Nw)

In [ ]:
plot_show_svd(data2)

## Test for different values of d

In [ ]:
test_method("cutin_loocv", data2, 1, 8, beta=.2, seed=0, overwrite=OVERWRITE)

## Test multiple times

In [ ]:
dmin, dmax = 2, 7
beta2 = 0.25
s1, s2 = test_multiple_times("cutin_loocv", data2, dmin, dmax, overwrite=OVERWRITE)
boxplots(s1 + beta2*(s1-s2))

## Validation of scoring measure

In [ ]:
dreal = 3
old_few2, old_many2, self_few2 = validation_scoring("cutin_loocv", data2, dreal, dmin, 
                                                 dmax, overwrite=OVERWRITE)
plot_correlation(old_many2, old_few2, self_few2)

In [ ]:
dreal = 7
old_few3, old_many3, self_few3 = validation_scoring("cutin_loocv", data2, dreal, dmin, 
                                                    dmax, overwrite=OVERWRITE)
plot_correlation(old_many3, old_few3, self_few3)

# Plots for papers

In [ ]:
FOLDER = os.path.join("..", "..", "20210301 Scenario generation", "figs")

def tikz_save(name):
    save(os.path.join(FOLDER, "{:s}.tikz".format(name)),
         axis_width='\\figurewidth', axis_height='\\figureheight',
         extra_axis_parameters=["xticklabel style={align=center}",
                                "yticklabel style={/pgf/number format/fixed,"+
                                "/pgf/number format/precision=3}",
                                "scaled y ticks=false"])

old_manys = [old_many1, old_many2]
old_fews = [old_few1, old_few2]
self_fews = [self_few1, self_few2]
betas = [beta1, beta2]
symbols = ["s", "D", "X"]
linestyles = ["-", "--"]

### Plot scores for both methods

In [ ]:
for i, (old_few, self_few, beta) in enumerate(zip(old_fews, self_fews, betas)):
    f, ax = plt.subplots()
    score = np.median(old_few + beta*(old_few - self_few), axis=0)
    dmax = old_few.shape[1] + 2 - dmin
    ax.plot(score, "o", c=(0, 0, 0))
    ax.set_xticks(np.arange(0, dmax-dmin+2))
    ax.set_xticklabels([r"Training\\set"] + 
                       [r"\$\dimension={:d}\$".format(d) for d in range(dmin, dmax+1)])
    ax.set_ylabel(r"\$\proposedmetric{\scenariosetgenerated}{\scenariosettest}{\scenarioset}\$")
    tikz_save("scores_{:d}".format(i+1))

### Plot "true" Wasserstein

In [ ]:
for i, (old_many, old_few, self_few) in enumerate(zip(old_manys, old_fews, self_fews)):
    f, ax = plt.subplots()
    dmax = old_few.shape[1] + 2 - dmin
    # (s) square = "true" Wasserstein
    # (D) diamond = empirical Wasserstein
    # (X) cross = "penalty"
    for x, s in zip([old_many, old_few, old_few-self_few], symbols):
        ax.plot(np.median(x, axis=0), marker=s, c=(0, 0, 0), linestyle="")
    ax.set_xticks(np.arange(0, dmax-dmin+2))
    ax.set_xticklabels([r"Training\\set"] + 
                       [r"\$\dimension={:d}\$".format(d) for d in range(dmin, dmax+1)])
    ax.set_ylabel("Wasserstein metric and penalty")
    tikz_save("wasserstein_{:d}".format(i+1))

### Correlation

In [ ]:
f, ax = plt.subplots()
# solid = lead vehicle decelerating
# dashed = cut-in
for old_many, old_few, self_few, ls in zip(old_manys, old_fews, self_fews, linestyles):
    beta = np.linspace(0, 1, 100)
    correlation = np.zeros(len(beta))
    for i in range(len(beta)):
        new_score = old_few + beta[i]*(old_few - self_few)
        correlation[i] = np.corrcoef(np.median(old_many, axis=0),
                                     np.median(new_score, axis=0))[0, 1]
    ax.plot(beta, correlation, c=(0, 0, 0), linestyle=ls)
ax.set_xlim(0, 1)
ax.set_xlabel(r"\$\penaltyweight\$")
ax.set_ylabel("Correlation")
tikz_save("correlation")